In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
from scipy.optimize import fsolve
import plotly.graph_objects as go
import plotly.express as px

## Loading Data

In [ ]:
filename = '../data/2020-03-10_QPD-Tilt-Calibration_Test1.csv'
col_names = ['theta_x', 'theta_y', 'qpd_x', 'qpd_y', 'qpd_sum']
df = pd.read_csv(filename, names=col_names)

### Check Data Validity

In [ ]:
df.head()

## Surface Fitting
Fit 2D Functions of form $q_x=f(\theta_x, \theta_y)$ and $q_y=f(\theta_x, \theta_y)$

e.g. using 3rd order polynomials:

$q_x = a_1.\theta_x^3 + a_2.\theta_y^3 + a_3.\theta_x^2 + a_4.\theta_y^2 + a_5.\theta_x + a_6.\theta_y + a_7$

$q_y = b_1.\theta_x^3 + b_2.\theta_y^3 + b_3.\theta_x^2 + b_4.\theta_y^2 + b_5.\theta_x + b_6.\theta_y + b_7$

In [ ]:
def surface_fitting(x, y, z):
    X = x.to_numpy().flatten()
    Y = y.to_numpy().flatten()
    Z = z.to_numpy().flatten()

    A = np.array([X**3, Y**3, X**2, Y**2, X, Y, X*0+1]).T
    #print(A.shape)

    coeff, r, rank, s = np.linalg.lstsq(A, Z, rcond=None)

    return coeff

In [ ]:
c = np.zeros([2, 7])
c[0, :] = surface_fitting(df['theta_x'], df['theta_y'], df['qpd_x'])
c[1, :] = surface_fitting(df['theta_x'], df['theta_y'], df['qpd_y'])
print(c)

### Plot the fitted surfaces

In [ ]:
def poly2Dreco(X, Y, c):
    return (c[0]*X**3 + c[1]*Y**3 + c[2]*X**2 + c[3]*Y**2 + c[4]*X + c[5]*Y + c[6])

def plot_fitted_surface(x, y, z, c, opts):
    
    grid_x, grid_y = np.mgrid[-0.5:0.5:100j, -0.4:0.9:100j]

    zfit = poly2Dreco(grid_x, grid_y, c)

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers'))


    fig.add_trace(go.Surface(z=zfit, x=grid_x, y=grid_y))
                    
    fig.update_layout(scene = plot_opts)
    fig.show()

Plot variation of QPD x-position with $\theta_x$ and $\theta_y$

In [ ]:
plot_opts = dict(
            xaxis_title='x-tilt angle (theta_x)',
            yaxis_title='y-tilt angle (theta_y)',
            zaxis_title='QPD x-position (qpd_x)')
plot_fitted_surface(df['theta_x'], df['theta_y'], df['qpd_x'], c[0, :], plot_opts)

Plot variation of QPD x-position with $\theta_x$ and $\theta_y$

In [ ]:
plot_opts = dict(
            xaxis_title='x-tilt angle (theta_x)',
            yaxis_title='y-tilt angle (theta_y)',
            zaxis_title='QPD y-position (qpd_y)')
plot_fitted_surface(df['theta_x'], df['theta_y'], df['qpd_y'], c[1, :], plot_opts)


## Calibration Function

Calculate $\theta_x$ and $\theta_y$ from $q_x$ an $q_y$ 

In [ ]:
def my_function(z, *data):
    '''
    z: contains theta values (to optimise)
    c: contains the (fixed) fitting coefficients describing the surfaces.
        c[0, :] contains the values for the function qpd_x(theta_x, theta_y)
        c[1, :] contains the values for the function qpd_y(theta_x, theta_y)
    q: contains the input qpd positions for the current iteration of the function
    '''
    c, q = data # tuple unpacking
    theta_x = z[0]
    theta_y = z[1]

    F = np.zeros_like(z)
    for i in range(0, 2):
        F[0] = c[i, 0]*theta_x**3 + c[i, 1]*theta_y**3 + c[i, 2]*theta_x**2 + c[i, 3]*theta_y**2 + c[i, 4]*theta_x + c[i, 5]*theta_y + c[i, 6] - q[i]

    return F

User enters QPD position for calculation:

In [ ]:
qpd_pos = np.array([0.1, 0.01])
print('input qpd position: ' + str(qpd_pos))

Use fsolve to calculate angles $\theta_x$ and $\theta_y$ 

In [ ]:
theta_guess = np.array([0, 0])
data = (c, qpd_pos)
thetas = fsolve(my_function, theta_guess, args=data)
print('calculated angles are ' + str(thetas) + ' degrees')

x_qpd_chk = poly2Dreco(thetas[0], thetas[1], c[0, ])
y_qpd_chk = poly2Dreco(thetas[0], thetas[1], c[1, ])

print('verification qpd position: ' + str([x_qpd_chk, y_qpd_chk]))

# Numerical Verification

In [ ]:
def plot_intersection(is_intersection, x_vec, y_vec, title_str):
    fig = go.Figure(data=go.Heatmap(
                    z=1*is_intersection, # convert bool --> int for plotting
                    x=x_vec,
                    y=y_vec))
    fig.update_layout(
                title=title_str,
                xaxis_title='x-tilt angle ($\\theta_x$)',
                yaxis_title='y-tilt angle ($\\theta_y$)',
                yaxis = dict(
                    scaleanchor = "x",
                    scaleratio = 1,)
                )
    fig.show()

In [ ]:
def calc_plane_surface_intersection(c, qpd_pos):
    x_vec = np.arange(-0.5, 0.5, 0.01)
    y_vec = np.arange(-0.4, 0.9, 0.01)
    grid_x, grid_y = np.meshgrid(x_vec, y_vec)

    for i in range(0, 2):
        z_qpd = poly2Dreco(grid_x, grid_y, c[i, ]) - qpd_pos[i] # z_qpd_x == 0 where qpd_x == qpd_pos[0]
        is_intersection = np.abs(z_qpd - 0) < 0.01

        if i == 0:
            title_str = 'QPD x-position intersection'
        elif i == 1:
            title_str = 'QPD y-position intersection'
        plot_intersection(is_intersection, x_vec, y_vec, title_str)

In [ ]:
calc_plane_surface_intersection(c, qpd_pos)